## load logistic and lorenz logs

In [38]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib notebook

In [39]:
# fig = plt.figure(figsize=(20,3.5))
# ax = plt.gca()

def plot_ts(pred):
    y_true, y_pred =pred
    ax = fig.add_subplot(1, 4, i+1)
    ax.plot(y_true[:100,seq_id],'g')
    ax.set_ylim([-20,20])
    ax = fig.add_subplot(2, 4, i+1)
    ax.plot(y_pred[:100,seq_id], 'r')
    ax.set_ylim([-20,20])
    ax.set_title(model)

### load logs for lorenz and print test error

In [40]:
time_stamp ="09-29-17-17-52-47"
exp='lorenz'
models =["LSTM","HOLSTM"] #'phased_lstm']#
hz_list= [8,16, 32]
lr_list =[1,2,3]



import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt



def read_config(fstream):
    d = {}
    for line in fstream:
        line.rstrip()
        tokens = line.split('\t')
        for token in tokens:
            try:
                key, val = token.split(":")
                d[key] = float(val)                
            except ValueError:
                'error prop'
    return d

def load_log(hz,lr, model):
    data_path = "/var/tmp/tensorRNN/log/"+exp+"/"+time_stamp+"/"+model+"/"+str(hz)+"/"+ "1e-"+str(lr)+"/"
    print(data_path)
    f_pred= "predict.npy"
    f_err = "config.out"
    pred =  np.load(data_path+f_pred )
    with open(data_path+""+f_err,'r') as fstream:
        config = read_config(fstream)
    return config, pred

errs = np.ones((len(models), len(hz_list)))
i=0
for model in models:
    j=0
    for hz in hz_list:
        k=0
        for lr in lr_list:
            print('load model {:s}, hidden_size'.format(model,hz))
            config, pred = load_log(hz, lr, model)
            print('train error:',config['train_error'],'test err:',config['test_error'])
            errs[i,j] =  config['test_error']
            k +=1
        j +=1
    i +=1


load model LSTM, hidden_size
/var/tmp/tensorRNN/log/lorenz/09-29-17-17-52-47/LSTM/8/1e-1/
train error: 0.392867 test err: 0.447346
load model LSTM, hidden_size
/var/tmp/tensorRNN/log/lorenz/09-29-17-17-52-47/LSTM/8/1e-2/
train error: 0.0461032 test err: 0.314293
load model LSTM, hidden_size
/var/tmp/tensorRNN/log/lorenz/09-29-17-17-52-47/LSTM/8/1e-3/
train error: 0.0428238 test err: 0.313333
load model LSTM, hidden_size
/var/tmp/tensorRNN/log/lorenz/09-29-17-17-52-47/LSTM/16/1e-1/
train error: 0.402023 test err: 0.373593
load model LSTM, hidden_size
/var/tmp/tensorRNN/log/lorenz/09-29-17-17-52-47/LSTM/16/1e-2/
train error: 0.040606 test err: 0.279543
load model LSTM, hidden_size
/var/tmp/tensorRNN/log/lorenz/09-29-17-17-52-47/LSTM/16/1e-3/
train error: 0.0497216 test err: 0.251171
load model LSTM, hidden_size
/var/tmp/tensorRNN/log/lorenz/09-29-17-17-52-47/LSTM/32/1e-1/
train error: 0.0419611 test err: 0.32877
load model LSTM, hidden_size
/var/tmp/tensorRNN/log/lorenz/09-29-17-17-52-47

### load seq-2-seq logs

In [34]:
import numpy as np
import sklearn 
from sklearn.metrics import mean_squared_error


def RMSE(y_pred, y_true):
    """RMSE"""
    t1 = np.linalg.norm(y_pred-y_true)**2 / np.size(y_true)
    return np.sqrt(t1)

data_path = "/var/tmp/tensorRNN/log/lorenz/09-28-17-14-12-08/LSTM/32/"
true, pred = np.load(data_path+'predict.npy')

diff = (true-pred)**2
diff = diff.flatten()
mse = np.sum(diff)/diff.size
val = np.sqrt(mse)
# true_reshape = np.reshape(true,(true.shape[0],-1))
# pred_reshape = np.reshape(pred,(pred.shape[0],-1))
# val = RMSE(true_reshape, pred_reshape)

print('rmse', val)
print(true[0,:10,0])
print(pred[0,:10,0])

(100, 96, 3)
rmse 0.277497787085
[ 0.42860848  0.43600047  0.44891748  0.47145543  0.51061326  0.57776546
  0.68838942  0.84905338  1.          0.95244271]
[ 0.43363535  0.43111402  0.45123914  0.4866333   0.54278558  0.62826616
  0.75636095  0.93828505  0.97565812  0.8407492 ]
